In [1]:
#importing libraries and frameworks

import tensorflow as tf
import string
import requests
import pandas as pd

In [2]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt') #fetching dataset

In [3]:
data=response.text.splitlines()
print(data[:10]) # 10 sample

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough"]


In [4]:
len(data)
#there is 2400 lines in this poetry

2400

In [5]:
# Total word number is:
len(" ".join(data))

91330

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [7]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data) #it's going to fit on the data in the forms of lines.

In [8]:
encoded_text=tokenizer.texts_to_sequences(data)
encoded_text[:10]
# These number actually comes from vocabulary
# it assigns number to each words

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [9]:
wc=tokenizer.word_counts
# word frequency

In [10]:
wi=tokenizer.word_index

In [11]:
print(f"Number of unique words and total vocab size: {len(tokenizer.word_counts)+1}")
vocab_size=len(tokenizer.word_counts)+1 # always adding plus one for tensorflow

Number of unique words and total vocab size: 1396


In [12]:
x=["play this song"]

In [13]:
tokenizer.texts_to_sequences(x)

[[241, 44, 409]]

### Prepare data for training

In [14]:
 encoded_text[:10] # 10 sample

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [15]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)):
            data_list.append(i[:j])


#### Paddding

In [16]:
max_length=20
#max length of line is 20

In [17]:
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre")
sequences[:5] # 5 sample

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 254,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 254,  21, 219],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 117,   8],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 117,   8,  80],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 117,   8,  80, 153]], dtype=int32)

In [18]:
sequences.shape

(14231, 20)

In [19]:
X=sequences[:,:-1]
y=sequences[:,-1]
print("X values")
print(X[:5]) # 5 sample
print("-"*30)
print("X values")
print(y[:5]) # 5 sample

X values
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  254]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 254
   21]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  117]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117
    8]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117   8
   80]]
------------------------------
X values
[ 21 219   8  80 153]


In [20]:
X.shape,y.shape

((14231, 19), (14231,))

In [21]:
y=to_categorical(y,num_classes=vocab_size)
#since unique word number is vocab_size, thus there is vocab_size classes
print(y[:5]) # 5 sample
print("Shape of y",y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of y (14231, 1396)


In [22]:
print("Shape of X",X.shape)
seq_length=X.shape[1]
seq_length

Shape of X (14231, 19)


19

#### Build Model
- We will build a simple LSTM model

In [23]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [26]:
model.fit(X,y,batch_size=32,epochs=250)

Epoch 1/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.0438 - loss: 5.9815
Epoch 2/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.0486 - loss: 5.2969
Epoch 3/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0550 - loss: 5.1133
Epoch 4/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0694 - loss: 4.9146
Epoch 5/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0771 - loss: 4.7660
Epoch 6/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0861 - loss: 4.6643
Epoch 7/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.0997 - loss: 4.5166
Epoch 8/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1120 - loss: 4.3941
Epoch 9/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.1182 - loss: 4.2970
Epoch 10/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.1368 - loss: 4.1617
Epoch 11/250
445/445 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.1481 - loss: 4.0775
Epoch 12/250
445/445 ━━━━━━━━━━━━━━━━━━━━

### Text Generation

In [27]:
text_lenght= 15 # 15 words per line

def generate_text(input_text, no_lines):
    general_text=[]
    for i in range(no_lines):
        text=[]
        for _ in range(text_lenght):
            encoded=tokenizer.texts_to_sequences([input_text])
            encoded=pad_sequences(encoded,maxlen=seq_length,padding="pre")
            y_pred=np.argmax(model.predict(encoded),axis=-1) # it will generate a word index, loop up into dictionary containing word index

            predicted_word=""
            for word,index in tokenizer.word_index.items():
                if index==y_pred:
                    predicted_word=word
                    break

            input_text=input_text +' '+ predicted_word
            text.append(predicted_word)

        input_text=text[-1]
        text=" ".join(text) # input text will be the last word of first created line
        general_text.append(text)

    return general_text

In [29]:
input_text="i love"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

['your precious heart i was her the things that all i will never be knocked',
 'baby i just want to make love to you happy new year turn down the',
 "scars of your love they leave me to work all you to this don't this",
 "ain't easy it's not meant to picture go only feel i never knew run for",
 'breaking your heart when i start to cry forgive me first be more if i',
 "want you to love me who knows why i love i'll i thought that i"]